In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import re
import os

import random
import json

import shutil

from tqdm import tqdm

from faker import Faker
import requests
import time

In [2]:
df = pd.read_csv("./home-and-garden.csv")

In [3]:
titles = df[~df['Title'].isna()].Title.values.tolist()
keywords = df[~df['Title'].isna()].Tags.values.tolist()
description = df[~df['Title'].isna()]['Body (HTML)'].transform(lambda x: x.replace("<p>", "")).transform(lambda x: x.replace("</p>", "")).values.tolist()  

In [4]:
keywords = [keywords_item for keywords_item in keywords]

In [5]:
images=[]
secImages = []
for title in titles:
    content = requests.get(f'https://api.unsplash.com/search/photos?query={title}&client_id=ZswFESXx_JnS_bzns7TNJJFNKa7VblY5xf_qyG3Gg-k')
    photoId = content.json()['results'][0]['id']
    secPhotoIds = []
    secPhotoIds.append(content.json()['results'][1]['id'])
    secPhotoIds.append(content.json()['results'][2]['id'])
    secPhotoIds.append(content.json()['results'][3]['id'])
    secPhotoIds = [f'https://source.unsplash.com/{photoId}/1620x1080' for photoId in secPhotoIds]
    time.sleep(1)
    images.append(f'https://source.unsplash.com/{photoId}/1620x1080')
    secImages.append(secPhotoIds)

KeyboardInterrupt: 

In [6]:
dummyDataProductInfo = []
for i in range(20):
    dummyDataProductInfo.append({
    "userId": "DrmLqVL3R7qgIPkSIolTzQ==",
    "price": str(random.randint(40,200)),
    "title": titles[i],
    "description": description[i],
    "keywords": keywords[i],
    "category": "furniture"})

out_file = open("dummyDataProductInfo.json", "w")
    
json.dump({"data":dummyDataProductInfo}, out_file)

In [7]:
for i in tqdm(range(20)):
    response = requests.get(images[i])
    time.sleep(1)

    file = open(f'./images/image{i}.jpg', "wb")
    file.write(response.content)
    file.close()
    
for i in tqdm(range(20)):
    for j in range(3):
        response = requests.get(secImages[i][j])
        time.sleep(1)

        file = open(f'./images/sec/image{i}_{j}.jpg', "wb")
        file.write(response.content)
        file.close()

 20%|██████████▊                                           | 4/20 [00:13<00:54,  3.38s/it]


KeyboardInterrupt: 

In [11]:
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

KEY = "" # the key of the user using the app

for i in tqdm(range(20)):
    resp = requests.post('http://127.0.0.1:8000/listing', json.dumps(dummyDataProductInfo[i]), headers={"Authorization": f'Bearer {KEY}'})
    data = resp.json()
    
    time.sleep(0.2)
    
    multipart_form_data = MultipartEncoder(fields={
        'image': (f'image{i}', open(f'./images/image{i}.jpg', 'rb')),
        'userId': str(data['userId']),
        'listingId': str(data['listingId']),
    })

    response = requests.patch('http://127.0.0.1:8000/listingMainImage', data=multipart_form_data,\
                              headers={'Content-Type': multipart_form_data.content_type, "Authorization": f'Bearer {KEY}'})
        

    for j in range(3):
        multipart_form_data = MultipartEncoder(fields={
            'image': (f'image{i}', open(f'./images/sec/image{i}_{j}.jpg', 'rb')),
            'userId': str(data['userId']),
            'listingId': str(data['listingId']),
        })

        response = requests.patch('http://127.0.0.1:8000/listingSecondaryImage', data=multipart_form_data,\
                                  headers={'Content-Type': multipart_form_data.content_type, "Authorization": f'Bearer {KEY}'})
        time.sleep(0.2)
  


100%|█████████████████████████████████████████████████████| 20/20 [01:18<00:00,  3.91s/it]
